In [1]:
import torch
import torch.functional as F
import torch.nn as nn
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from pprint import pprint
import re

In [2]:
torch.__version__

'2.4.1+cu121'

In [3]:
from os import linesep
text=""""Well, Prince, so Genoa and Lucca are now just family estates of the
Buonapartes. But I warn you, if you don't tell me that this means war,
if you still try to defend the infamies and horrors perpetrated by that
Antichrist--I really believe he is Antichrist--I will have nothing more
to do with you and you are no longer my friend, no longer my 'faithful
slave,' as you call yourself! But how do you do? I see I have frightened
you--sit down and tell me all the news."

It was in July, 1805, and the speaker was the well-known Anna Pavlovna
Scherer, maid of honor and favorite of the Empress Marya Fedorovna. With
these words she greeted Prince Vasili Kuragin, a man of high rank and
importance, who was the first to arrive at her reception. Anna Pavlovna
had had a cough for some days. She was, as she said, suffering from la
grippe; grippe being then a new word in St. Petersburg, used only by the
elite.

All her invitations without exception, written in French, and delivered
by a scarlet-liveried footman that morning, ran as follows:

"If you have nothing better to do, Count (or Prince), and if the
prospect of spending an evening with a poor invalid is not too terrible,
I shall be very charmed to see you tonight between 7 and 10--Annette
Scherer."

"Heavens! what a virulent attack!" replied the prince, not in the least
disconcerted by this reception. He had just entered, wearing an
embroidered court uniform, knee breeches, and shoes, and had stars on
his breast and a serene expression on his flat face. He spoke in that
refined French in which our grandfathers not only spoke but thought, and
with the gentle, patronizing intonation natural to a man of importance
who had grown old in society and at court. He went up to Anna Pavlovna,
kissed her hand, presenting to her his bald, scented, and shining head,
and complacently seated himself on the sofa.

"First of all, dear friend, tell me how you are. Set your friend's mind
at rest," said he without altering his tone, beneath the politeness and
affected sympathy of which indifference and even irony could be
discerned."""

filtered_text = re.sub('[^a-zA-Z0-9 \.\n]', '', text)

print(filtered_text)
words=[]
for (word) in filtered_text.split():
  if word.rstrip(".") not in words:
    words.append(word.rstrip("."))
  if (word[-1]=='.'):
    words.append('.')

para=filtered_text.split("\n\n")
print(len(para))

Well Prince so Genoa and Lucca are now just family estates of the
Buonapartes. But I warn you if you dont tell me that this means war
if you still try to defend the infamies and horrors perpetrated by that
AntichristI really believe he is AntichristI will have nothing more
to do with you and you are no longer my friend no longer my faithful
slave as you call yourself But how do you do I see I have frightened
yousit down and tell me all the news.

It was in July 1805 and the speaker was the wellknown Anna Pavlovna
Scherer maid of honor and favorite of the Empress Marya Fedorovna. With
these words she greeted Prince Vasili Kuragin a man of high rank and
importance who was the first to arrive at her reception. Anna Pavlovna
had had a cough for some days. She was as she said suffering from la
grippe grippe being then a new word in St. Petersburg used only by the
elite.

All her invitations without exception written in French and delivered
by a scarletliveried footman that morning ran as fo

In [4]:
stoi={s:i+1 for i,s in enumerate(words)}
stoi['.']=0
itos={i:s for s,i in stoi.items()}
pprint(itos)

{0: '.',
 1: 'Well',
 2: 'Prince',
 3: 'so',
 4: 'Genoa',
 5: 'and',
 6: 'Lucca',
 7: 'are',
 8: 'now',
 9: 'just',
 10: 'family',
 11: 'estates',
 12: 'of',
 13: 'the',
 14: 'Buonapartes',
 16: 'But',
 17: 'I',
 18: 'warn',
 19: 'you',
 20: 'if',
 21: 'dont',
 22: 'tell',
 23: 'me',
 24: 'that',
 25: 'this',
 26: 'means',
 27: 'war',
 28: 'still',
 29: 'try',
 30: 'to',
 31: 'defend',
 32: 'infamies',
 33: 'horrors',
 34: 'perpetrated',
 35: 'by',
 36: 'AntichristI',
 37: 'really',
 38: 'believe',
 39: 'he',
 40: 'is',
 41: 'will',
 42: 'have',
 43: 'nothing',
 44: 'more',
 45: 'do',
 46: 'with',
 47: 'no',
 48: 'longer',
 49: 'my',
 50: 'friend',
 51: 'faithful',
 52: 'slave',
 53: 'as',
 54: 'call',
 55: 'yourself',
 56: 'how',
 57: 'see',
 58: 'frightened',
 59: 'yousit',
 60: 'down',
 61: 'all',
 62: 'news',
 64: 'It',
 65: 'was',
 66: 'in',
 67: 'July',
 68: '1805',
 69: 'speaker',
 70: 'wellknown',
 71: 'Anna',
 72: 'Pavlovna',
 73: 'Scherer',
 74: 'maid',
 75: 'honor',
 76: 'fa

In [5]:
# Hyperparameter
block_size=5 # context_length: how many words do we take to predict the next one

# X and Y matrices to store the data for training
# X stores the half lines
# Y stores the next word
X,Y=[],[]
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for p in para:
  context=[0]*block_size

  for word in p.split():
    if (word[-1]=='.'):
      word=word[:-1]
      ix=stoi[word]
      X.append(context)
      Y.append(ix)
      print(' '.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix]

      word='.'
      ix=stoi[word]
      X.append(context)
      Y.append(ix)
      print(' '.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix]

    else:
      ix=stoi[word]
      X.append(context)
      Y.append(ix)
      print(' '.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix]


# Move data to GPU

X = torch.tensor(X).to(device)
Y = torch.tensor(Y).to(device)


. . . . . ---> Well
. . . . Well ---> Prince
. . . Well Prince ---> so
. . Well Prince so ---> Genoa
. Well Prince so Genoa ---> and
Well Prince so Genoa and ---> Lucca
Prince so Genoa and Lucca ---> are
so Genoa and Lucca are ---> now
Genoa and Lucca are now ---> just
and Lucca are now just ---> family
Lucca are now just family ---> estates
are now just family estates ---> of
now just family estates of ---> the
just family estates of the ---> Buonapartes
family estates of the Buonapartes ---> .
estates of the Buonapartes . ---> But
of the Buonapartes . But ---> I
the Buonapartes . But I ---> warn
Buonapartes . But I warn ---> you
. But I warn you ---> if
But I warn you if ---> you
I warn you if you ---> dont
warn you if you dont ---> tell
you if you dont tell ---> me
if you dont tell me ---> that
you dont tell me that ---> this
dont tell me that this ---> means
tell me that this means ---> war
me that this means war ---> if
that this means war if ---> you
this means war if you ---> st

In [6]:
X.shape, Y.shape, X.dtype, Y.dtype

(torch.Size([382, 5]), torch.Size([382]), torch.int64, torch.int64)

In [7]:
# Embedding layer for the context

emb_dim=2 # Hyperparameter
emb=torch.nn.Embedding(len(stoi),emb_dim).to(device)
print(emb)

# The emb contains weights which are vectors for each of the alphabet
print(emb.weight)

Embedding(219, 2)
Parameter containing:
tensor([[-0.6157, -0.1510],
        [ 0.1673,  1.7370],
        [ 1.1785,  0.3576],
        [-1.4603,  0.8752],
        [ 0.1656, -0.5529],
        [-0.4427,  1.7765],
        [ 1.0017,  2.5986],
        [-0.2841, -0.2656],
        [-0.5087,  1.1264],
        [-1.3530,  1.4586],
        [-0.6087,  0.2005],
        [-0.6741,  0.4103],
        [-0.3044, -0.0221],
        [ 0.4608, -0.9895],
        [ 1.2627,  1.1438],
        [-0.8116,  0.1054],
        [ 0.2663, -0.0907],
        [-0.0184, -0.1006],
        [ 2.2148,  0.9812],
        [ 0.8752,  0.3022],
        [ 0.5831,  0.1853],
        [-1.7159, -0.5469],
        [ 1.3902,  0.6697],
        [-2.1719,  1.2281],
        [ 0.1509, -0.2895],
        [-1.6336, -0.9729],
        [-0.0494, -1.1254],
        [-0.5813, -0.9846],
        [-0.7066,  1.6546],
        [-1.1040,  1.8525],
        [-0.8058, -0.3335],
        [-2.1184, -0.1623],
        [ 0.9311,  0.5151],
        [ 0.0515, -1.2209],
        

In [8]:
class Next_Word_Predictor(nn.Module):
  def __init__(self, block_size, vocab_size, emb_dim, hidden_dim):
    super().__init__()
    # Input size: vocab_size (the total number of characters in the vocabulary).
    # Output size: emb_dim (the size of the dense vector representation for each character).
    self.emb=nn.Embedding(vocab_size, emb_dim)

    # Input size: block_size * emb_dim
    # Output size: hidden_dim (the size of the hidden layer).
    self.linear1=nn.Linear(block_size*emb_dim, hidden_dim)

    # Input size: hidden_dim
    # Output size: vocab_size (the total number of words in the vocabulary).
    self.linear2=nn.Linear(hidden_dim, vocab_size)

  def forward(self, x):
    # I/P layer
    x = self.emb(x)
    x = x.view(x.shape[0], -1)
    # Hidden layer
    x = self.linear1(x)
    x = torch.relu(x)
    # Output layer
    x = self.linear2(x)
    return x


In [13]:
# Generate names from untrained model


model=Next_Word_Predictor(block_size, len(stoi), emb_dim, 100).to(device)
model=torch.compile(model)

def generate_next_word(model, itos, stoi, block_size, max_len=10):
  context=input("Enter the sentence: ")
  context=re.sub('[^a-zA-Z0-9 \.]', '', context)
  context = [stoi[word] for word in context.split()]

  # Adjust the length of context
  if len(context)<block_size:
      # Prepend zeros if the length is less than context_length
      context=[0]*(block_size-len(context))+context
  elif len(context)>block_size:
      # Take the last context_length words if the length is greater than context_length
      context=context[-block_size:]

  x=torch.tensor(context).view(1, -1).to(device)
  y_pred=model(x)
  ix=torch.distributions.categorical.Categorical(logits=y_pred).sample().item()
  word=itos[ix]
  return word

print(generate_next_word(model, itos, stoi, block_size))

Enter the sentence: you have
call


In [ ]:
for param_name, param in model.named_parameters():
    print(param_name, param.shape)

In [ ]:
import time
# Train the model

loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.AdamW(model.parameters(), lr=0.01)

# Mini-batch training
batch_size = 4096
print_every = 100
elapsed_time = []
for epoch in range(2000):
    start_time = time.time()
    for i in range(0, X.shape[0], batch_size):
        x = X[i:i+batch_size]
        y = Y[i:i+batch_size]
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        loss.backward()
        opt.step()
        opt.zero_grad()
    end_time = time.time()
    elapsed_time.append(end_time - start_time)
    if epoch % print_every == 0:
        print(epoch, loss.item())

In [ ]:
# Visualize the embedding

plot_emb(model.emb, itos)

In [ ]:
# Generate names from trained model

for i in range(1):
    print(generate_next_word(model, itos, stoi, block_size))